# CS Course Data Prep

In [119]:
data_path = "./data/csci.pdf"

import copy
import pickle

In [114]:
# course_logistics = []
# for course in copy.deepcopy(cs_courses):
#     del course["descriptionSearch"]
#     course_name = f"{course['department']} {course['number']} Section {course['section']}"
#     course_json_linearized = str(course)
#     res = f"{{course: {course_name}, {course_json_linearized[1:]}"
#     course_logistics.append(res)

# with open("./data/courses_prefixed_separated.txt", "w") as f:
#     for course in copy.deepcopy(cs_courses):
#         del course["descriptionSearch"]
        
#         course_name = f"{course['department']} {course['number']} Section {course['section']}"
#         f.write(f"{{course: {course_name}, ")
        
#         remainder_str = ", ".join([f"{k}: {v}" for k,v in course.items()])
#         f.write(f"{remainder_str}}}\n")

In [115]:
# course_logistics[0]

In [116]:
# course_descriptions = dict()

# for course in cs_courses:
#     course_name = f"{course['department']} {course['number']}"
#     if course_name not in course_descriptions:
#         course_descriptions[course_name] = course['descriptionSearch']

In [117]:
# course_descriptions

In [118]:
# with open("./data/courses_prefixed_separated.txt", "a") as f:
#     for k, v in course_descriptions.items():
#         f.write(f"{k}: {v}\n")

Each section entry should include:
- section name
- year
- semester
- courseID
- sectionType
- peoplesoftNumber
- classType
- instructors (nested dict)
- meetings 


Each section has repeated course info:
- descriptionSearch

- gradingBasisDesc

- classFormat (?)
- extraInfo (?)
- crossListing
- components

- titleLong
- titleShort

- courseAttributes (nested dict)

- classReqEval

- prereqs

- departmentNotes

- enrolmentPreferences


In [ ]:
course_segments = [
    ["descriptionSearch"],
    ["gradingBasisDesc"],
    ["titleLong", "titleShort"],
    ["classFormat", "extraInfo", "crossListing", "components"],
    ["courseAttributes"],
    ["classReqEval"],
    ["prereqs"],
    ["departmentNotes"],
    ["enrolmentPreferences"]
]

section_segments = ["year", "semester", "courseID", "sectionType", "peoplesoftNumber", "classType", "instructors", "meetings"]

In [ ]:
courses_processed = set()
with open("./data/courses_ultra_processed.txt", "w") as f:
    for section in copy.deepcopy(cs_courses):
        section_name = f"{section['department']} {section['number']} Section {section['section']}"
        course_name = f"{section['department']} {section['number']}"
        
        if course_name not in courses_processed:
            for segment_list in course_segments:
                # add info about this course
                course_line = f"{course_name} "
                course_line += ", ".join([f"{segment}: {section[segment]}" for segment in segment_list])
                course_line += "\n"
                f.write(course_line)
                courses_processed.add(course_name)
        
        section_line = f"{section_name} "
        section_line += ", ".join([f"{segment}: {section[segment]}" for segment in section_segments])
        section_line += "\n"
        f.write(section_line)

In [ ]:
import shutil
shutil.copyfile("./data/courses_ultra_processed.txt", "./data/everything_ultra_processed.txt")

with open("./data/cs_gold.txt", "r") as cs_gold:
    with open("./data/everything_ultra_processed.txt", "a") as f:
        for line in cs_gold:
            f.write(line)

In [ ]:
with open("./data/everything_ultra_processed.txt", "r") as f:
    for line in f:
        print(line)

    # print(max([(len(line), line) for line in f], key=lambda x:x[0]))

### Add sources as URLs & Pickle Data as Dict 

In [130]:
course_header_groups = [
    ["descriptionSearch"],
    ["gradingBasisDesc"],
    ["titleLong", "titleShort"],
    ["classFormat", "extraInfo", "crossListing", "components"],
    ["courseAttributes"],
    ["classReqEval"],
    ["prereqs"],
    ["departmentNotes"],
    ["enrolmentPreferences"]
]

section_headers = ["year", "semester", "courseID", "sectionType", "peoplesoftNumber", "classType", "instructors", "meetings"]

In [123]:
data = []  # list of tuples of the form (passage, source)
courses_processed = set()  # log courses processed so we don't repeat data across multiple sections

for section in copy.deepcopy(cs_courses):
    section_name = f"{section['department']} {section['number']} Section {section['section']}"
    course_name = f"{section['department']} {section['number']}"
    
    course_num = section['number']
    course_id = section['courseID']
    course_url = f"https://catalog.williams.edu/csci/detail/?strm=&cn={course_num}&crsid={course_id}&req_year=0"

    if course_name not in courses_processed:
        for header_group in course_header_groups:
            # add info about this course
            course_line = f"{course_name} "
            course_line += ", ".join([f"{header}: {section[header]}" for header in header_group])
            course_line += "\n"
            data.append((course_line, course_url))
            courses_processed.add(course_name)

    section_line = f"{section_name} "
    section_line += ", ".join([f"{segment}: {section[segment]}" for segment in section_segments])
    section_line += "\n"
    data.append((section_line, course_url))

In [131]:
with open("./data/cs_gold.txt", "r") as f:
    for line in f:
        data.append((line, "https://catalog.williams.edu/pdf/csci.pdf"))

In [132]:
with open("./data/everything_ultra_processed_w_sources.pkl", "wb") as f:
    pickle.dump(data, f)